# Get ready

First, download, import, prep packages and such. 

Then, check the file location and import the .csv files. Remove any with empty text fields. 

Save a data frame with all the texts and another with only those texts that mention the keywords of interest. 

In [1]:
%%capture

# installing necessary pdf conversion packages via pip
# the '%%capture' at the top of this cell suppresses the output (which is normally quite long and annoying looking). 
# You can remove or comment it out if you prefer to see the output. 
!pip install nltk
!pip install autocorrect        
!pip install pyspellchecker 
!pip install spacy -q
!python -m spacy download en_core_web_lg -q


In [11]:
import os                         # os is a module for navigating your machine (e.g., file directories).

import nltk                       # nltk stands for natural language tool kit and is useful for text-mining. 
from nltk import word_tokenize    # and some of its key functions
from nltk import sent_tokenize  
tokenizer = nltk.tokenize.punkt.PunktSentenceTokenizer()
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk.corpus import wordnet                    # Finally, things we need for lemmatising!
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer() 
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
nltk.download('averaged_perceptron_tagger')        # Like a POS-tagger...
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('webtext')
from nltk.corpus import webtext

import pandas as pd
pd.set_option('display.max_colwidth', 200)
import numpy as np
import statistics
import datetime
date = datetime.date.today()

from autocorrect import Speller   # things we need for spell checking
check = Speller(lang='en')
import codecs
import csv                        # csv is for importing and working with csv files

from collections import Counter

import statistics
import re                         # things we need for RegEx corrections
import matplotlib.pyplot as plt
import string 
import spacy 
from spacy.matcher import Matcher 
from spacy.tokens import Span 
from spacy import displacy 
nlp = spacy.load('en_core_web_lg')
nlp.max_length = 1500000 #or any large value, as long as you don't run out of RAM

import math 

English_punctuation = "-!\"#$%&()'*-–+,./:;<=>?@[\]^_`{|}~''“”"      # Things for removing punctuation, stopwords and empty strings
table_punctuation = str.maketrans('','', English_punctuation)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mzyssjkc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mzyssjkc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\mzyssjkc\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mzyssjkc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mzyssjkc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mzyssjkc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already u

In [5]:
print(os.listdir("..\\results")  )

files = []
def import_results(input):
    for f in os.listdir(input):
        f = pd.read_csv(input + '\\'+ f,encoding='latin1')
        files.append(f)
    output = pd.concat(files)
    return output

['ESHG2001abstractICHG.csv', 'ESHG2002Abstracts.csv', 'ESHG2003Abstracts.csv', 'ESHG2005Abstracts.csv', 'ESHG2006Abstracts.csv', 'ESHG2007Abstracts.csv', 'ESHG2008Abstracts.csv', 'ESHG2009Abstracts.csv', 'ESHG2010Abstracts.csv', 'ESHG2011Abstracts.csv', 'ESHG2012Abstracts.csv', 'ESHG2013Abstracts.csv', 'ESHG2014Abstracts.csv', 'ESHG2015Abstracts.csv', 'ESHG2016Abstractscleaned.csv']


In [6]:
all_results = import_results("..\\results")
len(all_results)

27413

In [7]:
no_null_texts = all_results[~all_results['Text'].isnull()]
len(no_null_texts)

26799

In [35]:
matched_texts = no_null_texts[no_null_texts['Text'].str.contains('autis|Autis|ASD|Asperger|asperger')]
len(matched_texts)

679

# Sent tokenisation

In [36]:
sentences  = [sent_tokenize(abstract) for abstract in matched_texts['Text'] ]
matched_texts['Sentence'] = sentences
matched_texts.head()

C:\Users\mzyssjkc\AppData\Local\Temp\ipykernel_1168\3272111604.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matched_texts['Sentence'] = sentences


,Title,Session_Code,Text,Email,Authors_and_Affiliations,Year,Author,Affiliations,Sentence
59,Genetic defects in sterol metabolism,S60.,Genetic defects in sterol metabolizing enzymes have recently emerged asimportant causes of dysmorphogenetic syndromes. They affect enzymesrequired for the removal of methyl groups at C4(NSHDL) th...,Fabian.Moebius@uibk.ac.at,F. MoebiusInstitute for Biochemical Pharmacology University of Innsbruck Innsbruck AustriaFabian.M,2001,NaN,NaN,"[Genetic defects in sterol metabolizing enzymes have recently emerged asimportant causes of dysmorphogenetic syndromes., They affect enzymesrequired for the removal of methyl groups at C4(NSHDL) ..."
150,Location of the first predisposing gene locus for Asperger syndrome on chromosome 1q21 22,C102.,Asperger syndrome (AS) was first described in 1944 by a Viennese physi cian Hans Asperger who reported a group of boys with autistic psychopa thy whose clinical features resembled autism with som...,irma.jarvela@hus.fi,E. Jarvela1 T. Ylisaukko oja2 T. Nieminen3 E. Kempas1 M. Auranen1 L. Peltonen1 1National Public Health Institute Helsinki Finland 2National Public Health Insitute Helsinki Finland 3Uni...,2001,NaN,NaN,[Asperger syndrome (AS) was first described in 1944 by a Viennese physi cian Hans Asperger who reported a group of boys with autistic psychopa thy whose clinical features resembled autism with so...
334,De novo der(5) identified as an interstitial insertion of chromosome 3 material by COBRA multi colour FISH,P0220.,bonn.deA 5 year old boy presented with ASD II bilateral cleft palate strabism bilat eral optic nerve coloboma sensorineural hearing loss bilateral inguinalherniae micropenis seizures and a...,hengels@meb.uni,Engels1 M. KreiÃÂ§ Nachtsheim1 A. Ehrbrecht1 S. Zahn1 R. Schubert1 G. Schwanitz1 C. Ergang1 M. van der Burg2 D. Hansmann3 A. K. Raap2 H. J. Tanke2 J. Wiegant2 1Institute of Human Gene...,2001,NaN,NaN,[ bonn.deA 5 year old boy presented with ASD II bilateral cleft palate strabism bilat eral optic nerve coloboma sensorineural hearing loss bilateral inguinalherniae micropenis seizures and ...
378,Duplication (4)(q31.1qter) in a newborn with suspi cious clinical diagnosis of Nijmegen breakage syndrome,P0267.,Nijmegen breakage syndrome (NBS) is a rare autosomal recessive condi tion characterised by progressive microcephaly early retardation ofgrowth chromosomal instability hypersensitivity to ionisi...,iveta.cernakova@hotmail.com,Cernakova1 M. Kvasnicova2 Z. Lovasova2 N. Badova3 E. Seemano va4 K. Spackova5 1Lab. experimental medicine Olomouc Czech Republic 2Dept. Clin. Genetics Banska Bystrica Slovakia 3Dept. ...,2001,NaN,NaN,[Nijmegen breakage syndrome (NBS) is a rare autosomal recessive condi tion characterised by progressive microcephaly early retardation ofgrowth chromosomal instability hypersensitivity to ionis...
385,A case of pure partial trisomy of 5q34 qter associated with asthma allergies and hyper IgE.,P0277.,Unbalanced chromosomal anomalies can lend information on the positionof genes and on the understanding of pathologic mechanisms of commondiseases. Thus one locus for asthma and allergic disease ...,suzanne.demczuk@muhc.mcgill.ca,Demczuk1 B. Dion1 P. Lepage2 B. R. Hadad3 T. Hudson2 V. M. Der Kaloustian4 1Montreal Children s Hospital Research Institute Montreal PQ Canada 2Genome Center Montreal General Hospital M...,2001,NaN,NaN,"[ Unbalanced chromosomal anomalies can lend information on the positionof genes and on the understanding of pathologic mechanisms of commondiseases., Thus one locus for asthma and allergic diseas..."


In [39]:
matched_texts = matched_texts.explode('Sentence')
matched_texts.head

<bound method NDFrame.head of                                                                                                             Title  \
59                                                                           Genetic defects in sterol metabolism   
59                                                                           Genetic defects in sterol metabolism   
59                                                                           Genetic defects in sterol metabolism   
59                                                                           Genetic defects in sterol metabolism   
59                                                                           Genetic defects in sterol metabolism   
...                                                                                                           ...   
1956   Gene expression patterns in relapsed B cell acute lymphoblastic leukemia and potential therapeutic targets   
1956   Gene expression patterns in

In [10]:
matched_sents = isolated_sent[~isolated_sent['tokenized_sents'].str.contains('autis|Autis|ASD|Asperger|asperger')]
len(matched_sents)

NameError: name 'isolated_sent' is not defined

In [ ]:
matched_sents

In [ ]:
search for word count - mutation - show frequency decreasing over time, increase in variant? gene change? 

disease also to decrease and condition or disorder to increase

In [ ]:
#for saving output
os.makedirs('folder/subfolder', exist_ok=True)  
df.to_csv('folder/subfolder/out.csv') 